In [ ]:
#Load the necessary libraries
import pandas as pd
import xlsxwriter
import openpyxl
import os

#Check the specified directory for all TEXT files and add them to a list
files = []
for file in os.listdir('C:\directory here\''):
    if file.endswith(".txt"):
        files.append(file)
        
#Use known discrepancy file to create a dictionary to reconcile against
known = pd.read_excel('C:\directory\excel.xlsx',sheet_name='Current', header=None, skiprows=1,
                     names=['B ID','Employee Name','Employee Group',['Error message'],etc])

known_dict = dict(zip(known['B ID'], known['Error message']))

#Create discrepancy fill object to format background to red
discrepancy_fill = openpyxl.styles.PatternFill(fill_type='solid', fgColor='FF0000')

"""
Iterate through the list of files, separate the current from terminated colleagues,
convert to an .xlsx file, delete the original .txt file, iterate through the known discrepancies,
open each .xlsx, format the background color of discrepancies and save
"""

if not files:
    pass
else:
    file_path = 'C:\\Directory\'
    
    for f in files:
        #Convert the pipe delimited file to a pandas dataframe and convert two columns to numeric
        df1 = pd.read_csv(file_path+f, sep='|')
        df1 = df1.applymap(lambda x: pd.to_numeric(x, errors='ignore'))
        
        #Separate the terminated employees from the employees colleagues and create two new dataframes
        nonterm_emp = df1[df1['Employment status'] != 'T'].index
        term_emp = df1[df1['Employment status'] == 'T'].index
        df2 = pd.DataFrame(data=df1, index=nonterm_emp)
        df3 = pd.DataFrame(data=df1, index=term_emp)
        
        #Define the path to save the output excel file and the name of that file
        out_name = f.replace('.txt', '.xlsx')
        
        #Create the excel writer object with the path and the output file name
        with pd.ExcelWriter(file_path+out_name, engine='xlsxwriter') as writer:
            df2.to_excel(writer, sheet_name='Employed')
            df3.to_excel(writer, sheet_name='Terminated')
            #Save the excel file
            writer.save()
            #Remove the original file
            os.remove(file_path+f)
            
        #Append the indices to the list when there is a match between the input and the known discrepancies file
        discrep_indices1 = []
        discrep_indices2 = []
        
        #Populate those discrepancy lists against the input file and the known discrepancies dictionary
        for i, bid in enumerate(df2['B ID']):
            if bid in known_dict.keys() and df2.iloc[i,df2.columns.get_loc('Error message')] == known_dict[bid]:
                discrep_indices1.append(i)
                
        for i, bid in enumerate(df3['B ID']):
            if bid in known_dict.keys() and df3.iloc[i,df3.columns.get_loc('Error message')] == known_dict[bid]:
                discrep_indices2.append(i)
        
        #Open each of the created excel files and load their worksheets
        for i in discrep_indices1:
            worksheet1.cell(row=i+2, column=1).fill = discrepancy_fill
            
        for j in discrep_indices2:
            worksheet2.cell(row=j+2, column=1).fill = discrepancy_fill
            
        #Final save of the output recon file
        workbook.save(file_path+out_name)